# Mentoría 'de cómo clasificar en géneros a las canciones'
## Práctico I : Análisis de features de audio

**Consideraciones:**
- Se evalúa el estilo y prolijidad del código.
- Se permite hacer trabajo extra, siempre y cuando las actividades básicas estén resueltas.

### Librerías

In [ ]:
!pip3 install spotipy
!pip3 install pandas

# Agregar las librerías extra que se utilicen en esta celda y la siguiente

### Dependencias y acceso al API

In [1]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '46b333d567314a89a6254b6c6b054be6'
client_secret = '9d922c3613e441518349dcf55f7d5853'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
#dir(sp)

### 1) Plantear una pregunta que quieran responder con estos datos

Las preguntas pueden ser por ejemplo: 
- ¿Por qué este artista destaca dentro de su género?
- ¿Por qué no lo hace?
- ¿Por qué un género musical es más escuchado que otro?

Sin embargo, pueden usar cualquier afirmación que hayan leído en internet y que se pueda responder con los datos este trabajo para validarla o rechazarla.

### Pregunta: Qué tan bien representan las features de audio de Spotify las características de canciones experimentales?

- Cómo se comportan las features ante la música experimental? Son capaces de representar lo que se escucha o dan resultados erróneos?



### 2) Obtener datos para responder la pregunta

Para esto hay que hacer pedidos a la API para obtener una lista de canciones de sus artistas, de las cuáles se obtienen sus features de audio como en este [ejemplo](https://developer.spotify.com/console/get-audio-features-track/?id=17VP4tofJ3evJbtY8Tk1Qi) de la versión web de la API. Una vez obtenidos los datos, seleccionar un subconjunto de 5 o 6 features de audio y agrupar los datos en un [DataFrame de pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

**Recomendación** : Usar un dataframe para su artista y otro para la competencia si es que eligierone esa opción, les va a ahorrar tiempo.

**Aclaración** : Si plantearon una pregunta que no requiera los datos de la consigna, obtengan esos datos.

In [28]:
# aux functions
def genres_by_artist_id(id):
    artist = sp.artist(id)
    genres = artist['genres']
    return genres

def songs_from_album_id(album_id): #album_id : str
    songs = []
    album = sp.album(album_id)
    for i in range(album['total_tracks']):
        track = {}
        track["song_name"] = album['tracks']['items'][i]['name']
        track["song_id"] = album['tracks']['items'][i]['id']
        track["album_name"] = album['name']
        track["album_id"] = album["id"]
        songs.append(track)
    return songs

def add_track(track_id, songs): #track_id:str, songs:List[dict]
    track = sp.track(track_id)
    audio_features = sp.audio_features(track_id)
    row = {}
    row["song_name"] =track['name']
    row["song_id"] = track['id']
    row["album_name"] = track['album']['name']
    row["album_id"] = album['album']['id']
    row["audio_features"] = audio_features[0]
    songs.append(row)
    return songs

def add_audio_features(songs): #songs: List[dict]
    for i in range(len(songs)):
        songs[i]["audio_features"] = sp.audio_features(songs[i]["song_id"])[0]
    return songs

In [73]:
aud= sp.audio_features("4Y2W4zKa3q72ztbkA0r8Va")[0]
aud
#genres_by_artist_id("4SQdUpG4f7UbkJG3cJ2Iyj")

{'acousticness': 0.507,
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4Y2W4zKa3q72ztbkA0r8Va',
 'danceability': 0.322,
 'duration_ms': 300187,
 'energy': 0.615,
 'id': '4Y2W4zKa3q72ztbkA0r8Va',
 'instrumentalness': 0.819,
 'key': 11,
 'liveness': 0.0677,
 'loudness': -10.774,
 'mode': 1,
 'speechiness': 0.0427,
 'tempo': 167.95,
 'time_signature': 4,
 'track_href': 'https://api.spotify.com/v1/tracks/4Y2W4zKa3q72ztbkA0r8Va',
 'type': 'audio_features',
 'uri': 'spotify:track:4Y2W4zKa3q72ztbkA0r8Va',
 'valence': 0.0361}

In [21]:
songs_df = pd.DataFrame(songs)
#pd.DataFrame(songs_df["audio_features"])
audio_features_df = pd.DataFrame(list(songs_df["audio_features"].values))
def track_by_feature(feature, value):
    return audio_features_df[audio_features_df[feature]==value]
track_by_feature("valence",0.032200)
audio_features_df.describe()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.329928,0.388700,187878.620000,0.497274,0.492349,5.880000,0.20056,-11.070700,0.500000,0.128838,110.085740,3.980000,0.196830
std,0.365397,0.176202,85566.344989,0.237744,0.354766,3.662496,0.14496,4.760213,0.505076,0.146931,35.773402,0.868731,0.152027
min,0.001520,0.148000,43524.000000,0.025700,0.000000,0.000000,0.05780,-23.257000,0.000000,0.031200,57.174000,1.000000,0.032200
25%,0.019750,0.231500,133184.750000,0.287750,0.140000,2.250000,0.10550,-13.389750,0.000000,0.041100,77.060750,4.000000,0.080775
50%,0.153150,0.359000,177771.000000,0.517000,0.542500,6.000000,0.14950,-10.680000,0.500000,0.059600,113.162000,4.000000,0.154000
75%,0.617750,0.522250,219339.500000,0.689250,0.844750,9.000000,0.26700,-7.213500,1.000000,0.162750,129.356750,4.000000,0.297250
max,0.993000,0.745000,548746.000000,0.895000,0.969000,11.000000,0.69800,-4.849000,1.000000,0.721000,191.563000,5.000000,0.674000


I chose 6 audio features to analyse Arca's music:
- **energy**: [float] Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
- **key**: [int] The key the track is in. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on.
- **tempo**: [float] The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
- **valence**:

### 3) Hacer un gráfico de estilo [radar chart](https://en.wikipedia.org/wiki/Radar_chart)  comparando a su artista o banda con 'la competencia'
En esta parte tienen que hacer los gráficos tomando como parámetros las features de audio que eligieron. Pueden usar la librería que quieran para hacer los gráficos.

**Recomendaciones** : 
- Hacer ambos gráficos en una misma figura.
- Antes de graficar, obtener los valores medios o medianos para los valores de las features. 
- Chequear los rangos de valores para cada feature antes de hacer los gráficos.

**Aclaración**: pueden usar otro tipo de gráfico si lo consideran particularmente útil para resolver su pregunta.

### 4) Escribir un informe

Este informe tiene que describir qué datos que obtuvieron, el volumen de estos (# de canciones del artista, # de canciones de la competencia, etc), una explicación de cómo estos se relacionan con la pregunta planteada, cómo ayudan a responderla y los resultados que obtuvieron. Usar la celda de abajo con formato markdown.




In [63]:
genres_by_artist_id("5bTeEWTzqpUapk0fZV8Clb")
#https://open.spotify.com/track/3ddSaaMuN43ddwF2qXF6Hz?si=tHOU37IhTAeT1oPutALBdw
#https://open.spotify.com/track/5kNGM33gtm1FGvH9KBipSZ?si=pDuyBgeORUC2tix1C35eYw
#https://open.spotify.com/track/6hfSrFbYHEshOOKfFzAb6P?si=TdniQbvkRXmStL6LAKobdg
#https://open.spotify.com/track/1IDi1OjPT8y7MmNflRYMHO?si=PChc4u5wR3SxblARhWp9SQ
sp.audio_features("4Y2W4zKa3q72ztbkA0r8Va")

[{'acousticness': 0.507,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4Y2W4zKa3q72ztbkA0r8Va',
  'danceability': 0.322,
  'duration_ms': 300187,
  'energy': 0.615,
  'id': '4Y2W4zKa3q72ztbkA0r8Va',
  'instrumentalness': 0.819,
  'key': 11,
  'liveness': 0.0677,
  'loudness': -10.774,
  'mode': 1,
  'speechiness': 0.0427,
  'tempo': 167.95,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/4Y2W4zKa3q72ztbkA0r8Va',
  'type': 'audio_features',
  'uri': 'spotify:track:4Y2W4zKa3q72ztbkA0r8Va',
  'valence': 0.0361}]

### Pregunta: ¿Qué tan bien representan las features de audio de Spotify las características de canciones experimentales?

- Cómo se comportan las features ante la música experimental? Son capaces de representar lo que se escucha o dan resultados erróneos?

Para esto armé un dataset pequeño con 30 canciones seleccionadas a mano, buscando representar distintos tipos de experimentación en lo sonoro. Elegí el tamaño del dataset ya que para comparar cada canción con las features hay que escucharlas a todas.
Entre las canciones hay cosas como
* Ruido blanco
* Noise y Harsh Noise 
* drone, avant-garde metal,atmospheric black metal ( Sun 0))), Deafheaven, The angelic process )
* Música predominantemente vocal, throat singing (Tanya Tagaq)
* experimental electronic (JASSS, Contrafacto 1IDi1OjPT8y7MmNflRYMHO Visceras musica)
* art pop, Experimental pop (Arca)
* synth punk (Deli girls)
* Ambient (Rainforest Spiritual Enslavement, Worms of the earth)